In [1]:
import pandas as pd

import psycopg2

import csv
import requests
import xml.etree.ElementTree as ET


## Загрузка данных из внешних таблиц в буферные внутренние в Greenplum

In [5]:
# Очистка буферных таблиц

with psycopg2.connect(
        host='172.17.1.32',
        user='gpadmin',
        password='',
        dbname='wave15_team_a',
        port=5432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('TRUNCATE gp_clients_buff, gp_securities_buff, gp_currencies_buff, gp_security_transactions_buff, gp_bank_transactions_buff RESTART IDENTITY CASCADE')

In [ ]:
# Загрузка данных из внешних таблиц в HDFS во внутренние буферные в Greenplum

with psycopg2.connect(
        host='172.17.1.32',
        user='gpadmin',
        password='',
        dbname='wave15_team_a',
        port=5432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        #next(f)
        #cur.copy_from('gp_ext_clients', 'gp_clients_buff', columns=['client_id', 'client_name', 'client_email', 'client_phone', 'client_address'], sep=";")
        cur.execute('INSERT INTO gp_clients_buff SELECT * FROM ext_clients')
        cur.execute('INSERT INTO gp_securities_buff SELECT * FROM ext_securities')
        cur.execute('INSERT INTO gp_currencies_buff SELECT * FROM ext_currencies')
        cur.execute('INSERT INTO gp_security_transactions_buff SELECT * FROM ext_security_transactions')
        cur.execute('INSERT INTO gp_bank_transactions_buff SELECT * FROM ext_bank_transactions')

## Перемещение данных из буферных таблиц в базовые с помощью хранимых функций

In [ ]:
# Перемещение данных из буферных таблиц в базовые с помощью хранимых функций

with psycopg2.connect(
        host='172.17.1.32',
        user='gpadmin',
        password='',
        dbname='wave15_team_a',
        port=5432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        #next(f)
        #cur.copy_from('gp_ext_clients', 'gp_clients_buff', columns=['client_id', 'client_name', 'client_email', 'client_phone', 'client_address'], sep=";")
        cur.execute('SELECT gp_buff_to_base_transfer_clients()')
        cur.execute('SELECT gp_buff_to_base_transfer_securities()')
        cur.execute('SELECT gp_buff_to_base_transfer_currencies()')
        cur.execute('SELECT gp_buff_to_base_transfer_security_transactions()')
        cur.execute('SELECT gp_buff_to_base_transfer_bank_transactions()')

## Вспомогательные скрипты

In [6]:
# Очистка базовых таблиц

with psycopg2.connect(
        host='172.17.1.32',
        user='gpadmin',
        password='',
        dbname='wave15_team_a',
        port=5432) as conn:
    with conn.cursor() as cur:
        cur.execute('SET search_path TO ashkinadzi')
        cur.execute('TRUNCATE gp_clients_base, gp_securities_base, gp_currencies_base, gp_security_transactions_base, gp_bank_transactions_base RESTART IDENTITY CASCADE')